In [1]:
import os
import sys
import json
import pyccl
import scipy
import numpy
from itertools import product
from matplotlib import pyplot
import matplotlib.lines as mlines
from matplotlib.gridspec import GridSpec

In [2]:
# Path
TAG = 'Y1'
LABEL = 'CELL'
PATH = '/pscratch/sd/y/yhzhang/LimberCloud'
sys.path.insert(0, os.path.join(PATH, 'PYTHON'))
PLOT_PATH = os.path.join(PATH, 'PLOT', LABEL, TAG)

# Folder
FOLDER = '/global/cfs/cdirs/lsst/groups/MCP/CosmoCloud/LimberCloud/'
PLOT_FOLDER = os.path.join(FOLDER, 'PLOT', LABEL, TAG)
DATA_FOLDER = os.path.join(FOLDER, 'DATA', TAG)
INFO_FOLDER = os.path.join(FOLDER, 'INFO')

In [3]:
# Cosmology
with open(os.path.join(INFO_FOLDER, 'COSMOLOGY.json'), 'r') as file:
    COSMOLOGY = json.load(file)

CCL = pyccl.Cosmology(
    h=COSMOLOGY['H'],
    w0=COSMOLOGY['W0'],
    wa=COSMOLOGY['WA'], 
    n_s=COSMOLOGY['NS'], 
    A_s=COSMOLOGY['AS'],
    m_nu=COSMOLOGY['M_NU'],  
    Neff=COSMOLOGY['N_EFF'],
    Omega_b=COSMOLOGY['OMEGA_B'], 
    Omega_k=COSMOLOGY['OMEGA_K'], 
    Omega_c=COSMOLOGY['OMEGA_CDM'], 
    mass_split='single', matter_power_spectrum='halofit', transfer_function='boltzmann_camb',
    extra_parameters={'camb': {'kmax': 50, 'lmax': 5000, 'halofit_version': 'mead2020_feedback', 'HMCode_logT_AGN': 7.8}}
)

In [4]:
# Angular power spectra

def w_m_N(chi_data, chi_grid, phi_m_grid, type):
    phi_m = scipy.interpolate.interp1d(chi_grid, phi_m_grid, kind=type)
    return phi_m(chi_data) / chi_data

def w_m_S(chi_data, chi_grid, phi_m_grid, redshift_grid, type):
    data_size = len(chi_data)
    w = numpy.zeros(data_size)
    phi_m = scipy.interpolate.interp1d(chi_grid, phi_m_grid, kind=type)
    scale = scipy.interpolate.interp1d(chi_grid, 1 / (1 + redshift_grid), kind=type)
    
    for k in range(data_size):
        def function(chi_prime):
            return phi_m(chi_prime) * (chi_prime - chi_data[k]) / chi_prime
        w[k] = scipy.integrate.fixed_quad(func=function, a=chi_data[k], b=chi_grid.max(), n=500)[0] / scale(chi_data[k])
    return w

def c_a_b_SS(factor, amplitude, phi_a_grid, phi_b_grid, chi_grid, power_grid, redshift_grid, type):
    
    power = scipy.interpolate.interp1d(chi_grid, power_grid, kind=type)
    
    def function(chi_data):
        return power(chi_data) * w_m_S(chi_data, chi_grid, phi_a_grid, redshift_grid, type) * w_m_S(chi_data, chi_grid, phi_b_grid, redshift_grid, type)
    
    return factor * amplitude * scipy.integrate.fixed_quad(func=function, a=chi_grid.min(), b=chi_grid.max(), n=500)[0]

def c_a_b_NS(factor, amplitude, phi_a_grid, phi_b_grid, chi_grid, power_grid, redshift_grid, type):
    
    power = scipy.interpolate.interp1d(chi_grid, power_grid, kind=type)
    
    def function(chi_data):
        return power(chi_data) * w_m_N(chi_data, chi_grid, phi_a_grid, type) * w_m_S(chi_data, chi_grid, phi_b_grid, redshift_grid, type)
    
    return factor * amplitude * scipy.integrate.fixed_quad(func=function, a=chi_grid.min(), b=chi_grid.max(), n=500)[0]

def c_a_b_NN(factor, amplitude, phi_a_grid, phi_b_grid, chi_grid, power_grid, type):
    
    power = scipy.interpolate.interp1d(chi_grid, power_grid, kind=type)
    
    def function(chi_data):
        return power(chi_data) * w_m_N(chi_data, chi_grid, phi_a_grid, type) * w_m_N(chi_data, chi_grid, phi_b_grid, type)
    
    return factor * amplitude * scipy.integrate.fixed_quad(func=function, a=chi_grid.min(), b=chi_grid.max(), n=500)[0]

In [5]:
# Load in Data
SOURCE = numpy.load(os.path.join(DATA_FOLDER, 'lsst_source_bins.npy'), allow_pickle=True).item()

# Size
GRID_SIZE = 350
SOURCE_BIN_SIZE = len(SOURCE['bins'])
SOURCE_REDSHIFT = SOURCE['redshift_range']

# Grid
Z1 = SOURCE_REDSHIFT.min()
Z2 = SOURCE_REDSHIFT.max()
Z_GRID = numpy.linspace(Z1, Z2, GRID_SIZE + 1)

SOURCE_PSI_GRID = numpy.zeros((SOURCE_BIN_SIZE, GRID_SIZE + 1))
for BIN_INDEX in range(SOURCE_BIN_SIZE):
    SOURCE_PSI_GRID[BIN_INDEX, :] = numpy.interp(x=Z_GRID, xp=SOURCE_REDSHIFT, fp=SOURCE['bins'][BIN_INDEX])
    SOURCE_PSI_GRID[BIN_INDEX, :] = SOURCE_PSI_GRID[BIN_INDEX, :] / scipy.integrate.trapezoid(x=Z_GRID, y=SOURCE_PSI_GRID[BIN_INDEX, :])

# Alignment
with open(os.path.join(INFO_FOLDER, 'ALIGNMENT.json'), 'r') as file:
    ALIGNMENT = numpy.array(json.load(file)['A'])

In [6]:
# Phi
A_GRID = 1 / (1 + Z_GRID)
CHI_GRID = pyccl.background.comoving_radial_distance(cosmo=CCL, a=A_GRID)
SOURCE_PHI_GRID = SOURCE_PSI_GRID * CCL.h_over_h0(a=A_GRID) * COSMOLOGY['H'] * 100000 / scipy.constants.c

# Multipole
ELL1 = 20
ELL2 = 2000
ELL_SIZE = 20
ELL_GRID = numpy.geomspace(ELL1, ELL2, ELL_SIZE + 1)

CHI_MESH, ELL_MESH = numpy.meshgrid(CHI_GRID, ELL_GRID)
SCALE_GRID = numpy.nan_to_num(numpy.divide(ELL_MESH + 1/2, CHI_MESH, out=numpy.zeros((ELL_SIZE + 1, GRID_SIZE + 1)) + numpy.inf, where=CHI_MESH > 0))

# Power
POWER_GRID = numpy.zeros((ELL_SIZE + 1, GRID_SIZE + 1))
for GRID_INDEX in range(GRID_SIZE + 1):
    POWER_GRID[:,GRID_INDEX] = pyccl.power.nonlin_matter_power(cosmo=CCL, k=SCALE_GRID[:,GRID_INDEX], a=A_GRID[GRID_INDEX])

# Amplitude
AMPLITUDE = 3 / 2 * COSMOLOGY['OMEGA_M'] * (COSMOLOGY['H'] * 100000 / scipy.constants.c) ** 2
AMPLITUDE_SS = AMPLITUDE ** 2
AMPLITUDE_SI = AMPLITUDE
AMPLITUDE_IS = AMPLITUDE
AMPLITUDE_II = 1

# Factor
FACTOR_SS = (1 + 3 / (2 * ELL_GRID + 1)) * (1 + 1 / (2 * ELL_GRID + 1)) * (1 - 1 / (2 * ELL_GRID + 1)) * (1 - 3 / (2 * ELL_GRID + 1))
FACTOR_SI = numpy.sqrt((1 + 3 / (2 * ELL_GRID + 1)) * (1 + 1 / (2 * ELL_GRID + 1)) * (1 - 1 / (2 * ELL_GRID + 1)) * (1 - 3 / (2 * ELL_GRID + 1)))
FACTOR_IS = numpy.sqrt((1 + 3 / (2 * ELL_GRID + 1)) * (1 + 1 / (2 * ELL_GRID + 1)) * (1 - 1 / (2 * ELL_GRID + 1)) * (1 - 3 / (2 * ELL_GRID + 1)))
FACTOR_II = numpy.ones(ELL_SIZE + 1)

In [ ]:
# C_ELL_EE
C_CCL_SS = numpy.zeros((SOURCE_BIN_SIZE, SOURCE_BIN_SIZE, ELL_SIZE + 1))
C_CCL_SI = numpy.zeros((SOURCE_BIN_SIZE, SOURCE_BIN_SIZE, ELL_SIZE + 1))
C_CCL_IS = numpy.zeros((SOURCE_BIN_SIZE, SOURCE_BIN_SIZE, ELL_SIZE + 1))
C_CCL_II = numpy.zeros((SOURCE_BIN_SIZE, SOURCE_BIN_SIZE, ELL_SIZE + 1))

C_DATA1_SS = numpy.zeros((SOURCE_BIN_SIZE, SOURCE_BIN_SIZE, ELL_SIZE + 1))
C_DATA1_SI = numpy.zeros((SOURCE_BIN_SIZE, SOURCE_BIN_SIZE, ELL_SIZE + 1))
C_DATA1_IS = numpy.zeros((SOURCE_BIN_SIZE, SOURCE_BIN_SIZE, ELL_SIZE + 1))
C_DATA1_II = numpy.zeros((SOURCE_BIN_SIZE, SOURCE_BIN_SIZE, ELL_SIZE + 1))

C_DATA2_SS = numpy.zeros((SOURCE_BIN_SIZE, SOURCE_BIN_SIZE, ELL_SIZE + 1))
C_DATA2_SI = numpy.zeros((SOURCE_BIN_SIZE, SOURCE_BIN_SIZE, ELL_SIZE + 1))
C_DATA2_IS = numpy.zeros((SOURCE_BIN_SIZE, SOURCE_BIN_SIZE, ELL_SIZE + 1))
C_DATA2_II = numpy.zeros((SOURCE_BIN_SIZE, SOURCE_BIN_SIZE, ELL_SIZE + 1))

C_DATA3_SS = numpy.zeros((SOURCE_BIN_SIZE, SOURCE_BIN_SIZE, ELL_SIZE + 1))
C_DATA3_SI = numpy.zeros((SOURCE_BIN_SIZE, SOURCE_BIN_SIZE, ELL_SIZE + 1))
C_DATA3_IS = numpy.zeros((SOURCE_BIN_SIZE, SOURCE_BIN_SIZE, ELL_SIZE + 1))
C_DATA3_II = numpy.zeros((SOURCE_BIN_SIZE, SOURCE_BIN_SIZE, ELL_SIZE + 1))

for (BIN_INDEX1, BIN_INDEX2) in product(range(SOURCE_BIN_SIZE), range(SOURCE_BIN_SIZE)):
    TRACER1 = pyccl.tracers.WeakLensingTracer(cosmo=CCL, dndz=[Z_GRID, SOURCE_PSI_GRID[BIN_INDEX1, :]], has_shear=True, ia_bias=None, use_A_ia=False, n_samples=GRID_SIZE + 1)
    TRACER2 = pyccl.tracers.WeakLensingTracer(cosmo=CCL, dndz=[Z_GRID, SOURCE_PSI_GRID[BIN_INDEX2, :]], has_shear=True, ia_bias=None, use_A_ia=False, n_samples=GRID_SIZE + 1)
    C_CCL_SS[BIN_INDEX1, BIN_INDEX2,:] = pyccl.cells.angular_cl(cosmo=CCL, tracer1=TRACER1, tracer2=TRACER2, ell=ELL_GRID, p_of_k_a='delta_matter:delta_matter', l_limber=-1, limber_max_error=0.001, limber_integration_method='qag_quad', p_of_k_a_lin='delta_matter:delta_matter', return_meta=False)
    
    TRACER1 = pyccl.tracers.WeakLensingTracer(cosmo=CCL, dndz=[Z_GRID, SOURCE_PSI_GRID[BIN_INDEX1, :]], has_shear=True, ia_bias=None, use_A_ia=False, n_samples=GRID_SIZE + 1)
    TRACER2 = pyccl.tracers.WeakLensingTracer(cosmo=CCL, dndz=[Z_GRID, SOURCE_PSI_GRID[BIN_INDEX2, :]], has_shear=False, ia_bias=[Z_GRID, ALIGNMENT], use_A_ia=False, n_samples=GRID_SIZE + 1)
    C_CCL_SI[BIN_INDEX1, BIN_INDEX2,:] = pyccl.cells.angular_cl(cosmo=CCL, tracer1=TRACER1, tracer2=TRACER2, ell=ELL_GRID, p_of_k_a='delta_matter:delta_matter', l_limber=-1, limber_max_error=0.001, limber_integration_method='qag_quad', p_of_k_a_lin='delta_matter:delta_matter', return_meta=False)
    
    TRACER1 = pyccl.tracers.WeakLensingTracer(cosmo=CCL, dndz=[Z_GRID, SOURCE_PSI_GRID[BIN_INDEX1, :]], has_shear=False, ia_bias=[Z_GRID, ALIGNMENT], use_A_ia=False, n_samples=GRID_SIZE + 1)
    TRACER2 = pyccl.tracers.WeakLensingTracer(cosmo=CCL, dndz=[Z_GRID, SOURCE_PSI_GRID[BIN_INDEX2, :]], has_shear=True, ia_bias=None, use_A_ia=False, n_samples=GRID_SIZE + 1)
    C_CCL_IS[BIN_INDEX1, BIN_INDEX2,:] = pyccl.cells.angular_cl(cosmo=CCL, tracer1=TRACER1, tracer2=TRACER2, ell=ELL_GRID, p_of_k_a='delta_matter:delta_matter', l_limber=-1, limber_max_error=0.001, limber_integration_method='qag_quad', p_of_k_a_lin='delta_matter:delta_matter', return_meta=False)
    
    TRACER1 = pyccl.tracers.WeakLensingTracer(cosmo=CCL, dndz=[Z_GRID, SOURCE_PSI_GRID[BIN_INDEX1, :]], has_shear=False, ia_bias=[Z_GRID, ALIGNMENT], use_A_ia=False, n_samples=GRID_SIZE + 1)
    TRACER2 = pyccl.tracers.WeakLensingTracer(cosmo=CCL, dndz=[Z_GRID, SOURCE_PSI_GRID[BIN_INDEX2, :]], has_shear=False, ia_bias=[Z_GRID, ALIGNMENT], use_A_ia=False, n_samples=GRID_SIZE + 1)
    C_CCL_II[BIN_INDEX1, BIN_INDEX2,:] = pyccl.cells.angular_cl(cosmo=CCL, tracer1=TRACER1, tracer2=TRACER2, ell=ELL_GRID, p_of_k_a='delta_matter:delta_matter', l_limber=-1, limber_max_error=0.001, limber_integration_method='qag_quad', p_of_k_a_lin='delta_matter:delta_matter', return_meta=False)
    
    for ELL_INDEX in range(ELL_SIZE + 1):
        C_DATA1_SS[BIN_INDEX1, BIN_INDEX2, ELL_INDEX] = c_a_b_SS(factor=FACTOR_SS[ELL_INDEX], amplitude=AMPLITUDE_SS, phi_a_grid=SOURCE_PHI_GRID[BIN_INDEX1,:], phi_b_grid=SOURCE_PHI_GRID[BIN_INDEX2,:], chi_grid=CHI_GRID, power_grid=POWER_GRID[ELL_INDEX,:], redshift_grid=Z_GRID, type='slinear')
        C_DATA1_SI[BIN_INDEX1, BIN_INDEX2, ELL_INDEX] = c_a_b_NS(factor=FACTOR_SI[ELL_INDEX], amplitude=AMPLITUDE_SI, phi_a_grid=SOURCE_PHI_GRID[BIN_INDEX1,:], phi_b_grid=SOURCE_PHI_GRID[BIN_INDEX2,:], chi_grid=CHI_GRID, power_grid=POWER_GRID[ELL_INDEX,:] * ALIGNMENT, redshift_grid=Z_GRID, type='slinear')
        C_DATA1_IS[BIN_INDEX1, BIN_INDEX2, ELL_INDEX] = c_a_b_NS(factor=FACTOR_IS[ELL_INDEX], amplitude=AMPLITUDE_IS, phi_a_grid=SOURCE_PHI_GRID[BIN_INDEX1,:], phi_b_grid=SOURCE_PHI_GRID[BIN_INDEX2,:], chi_grid=CHI_GRID, power_grid=POWER_GRID[ELL_INDEX,:] * ALIGNMENT, redshift_grid=Z_GRID, type='slinear')
        C_DATA1_II[BIN_INDEX1, BIN_INDEX2, ELL_INDEX] = c_a_b_NN(factor=FACTOR_II[ELL_INDEX], amplitude=AMPLITUDE_II, phi_a_grid=SOURCE_PHI_GRID[BIN_INDEX1,:], phi_b_grid=SOURCE_PHI_GRID[BIN_INDEX2,:], chi_grid=CHI_GRID, power_grid=POWER_GRID[ELL_INDEX,:] * ALIGNMENT ** 2, type='slinear')
        
        C_DATA2_SS[BIN_INDEX1, BIN_INDEX2, ELL_INDEX] = c_a_b_SS(factor=FACTOR_SS[ELL_INDEX], amplitude=AMPLITUDE_SS, phi_a_grid=SOURCE_PHI_GRID[BIN_INDEX1,:], phi_b_grid=SOURCE_PHI_GRID[BIN_INDEX2,:], chi_grid=CHI_GRID, power_grid=POWER_GRID[ELL_INDEX,:], redshift_grid=Z_GRID, type='quadratic')
        C_DATA2_SI[BIN_INDEX1, BIN_INDEX2, ELL_INDEX] = c_a_b_NS(factor=FACTOR_SI[ELL_INDEX], amplitude=AMPLITUDE_SI, phi_a_grid=SOURCE_PHI_GRID[BIN_INDEX1,:], phi_b_grid=SOURCE_PHI_GRID[BIN_INDEX2,:], chi_grid=CHI_GRID, power_grid=POWER_GRID[ELL_INDEX,:] * ALIGNMENT, redshift_grid=Z_GRID, type='quadratic')
        C_DATA2_IS[BIN_INDEX1, BIN_INDEX2, ELL_INDEX] = c_a_b_NS(factor=FACTOR_IS[ELL_INDEX], amplitude=AMPLITUDE_IS, phi_a_grid=SOURCE_PHI_GRID[BIN_INDEX1,:], phi_b_grid=SOURCE_PHI_GRID[BIN_INDEX2,:], chi_grid=CHI_GRID, power_grid=POWER_GRID[ELL_INDEX,:] * ALIGNMENT, redshift_grid=Z_GRID, type='quadratic')
        C_DATA2_II[BIN_INDEX1, BIN_INDEX2, ELL_INDEX] = c_a_b_NN(factor=FACTOR_II[ELL_INDEX], amplitude=AMPLITUDE_II, phi_a_grid=SOURCE_PHI_GRID[BIN_INDEX1,:], phi_b_grid=SOURCE_PHI_GRID[BIN_INDEX2,:], chi_grid=CHI_GRID, power_grid=POWER_GRID[ELL_INDEX,:] * ALIGNMENT ** 2, type='quadratic')
        
        C_DATA3_SS[BIN_INDEX1, BIN_INDEX2, ELL_INDEX] = c_a_b_SS(factor=FACTOR_SS[ELL_INDEX], amplitude=AMPLITUDE_SS, phi_a_grid=SOURCE_PHI_GRID[BIN_INDEX1,:], phi_b_grid=SOURCE_PHI_GRID[BIN_INDEX2,:], chi_grid=CHI_GRID, power_grid=POWER_GRID[ELL_INDEX,:], redshift_grid=Z_GRID, type='cubic')
        C_DATA3_SI[BIN_INDEX1, BIN_INDEX2, ELL_INDEX] = c_a_b_NS(factor=FACTOR_SI[ELL_INDEX], amplitude=AMPLITUDE_SI, phi_a_grid=SOURCE_PHI_GRID[BIN_INDEX1,:], phi_b_grid=SOURCE_PHI_GRID[BIN_INDEX2,:], chi_grid=CHI_GRID, power_grid=POWER_GRID[ELL_INDEX,:] * ALIGNMENT, redshift_grid=Z_GRID, type='cubic')
        C_DATA3_IS[BIN_INDEX1, BIN_INDEX2, ELL_INDEX] = c_a_b_NS(factor=FACTOR_IS[ELL_INDEX], amplitude=AMPLITUDE_IS, phi_a_grid=SOURCE_PHI_GRID[BIN_INDEX1,:], phi_b_grid=SOURCE_PHI_GRID[BIN_INDEX2,:], chi_grid=CHI_GRID, power_grid=POWER_GRID[ELL_INDEX,:] * ALIGNMENT, redshift_grid=Z_GRID, type='cubic')
        C_DATA3_II[BIN_INDEX1, BIN_INDEX2, ELL_INDEX] = c_a_b_NN(factor=FACTOR_II[ELL_INDEX], amplitude=AMPLITUDE_II, phi_a_grid=SOURCE_PHI_GRID[BIN_INDEX1,:], phi_b_grid=SOURCE_PHI_GRID[BIN_INDEX2,:], chi_grid=CHI_GRID, power_grid=POWER_GRID[ELL_INDEX,:] * ALIGNMENT ** 2, type='cubic')

In [ ]:
# Projection
from PROJECTION import NN, NS, SS

C_DATA_SS = SS.spectra(
    factor=numpy.array(FACTOR_SS, dtype=numpy.float64), 
    amplitude=numpy.array(AMPLITUDE_SS, dtype=numpy.float64), 
    chi_grid=numpy.array(CHI_GRID, dtype=numpy.float64), 
    power_grid=numpy.array(POWER_GRID, dtype=numpy.float64), 
    redshift_grid=numpy.array(Z_GRID, dtype=numpy.float64),
    phi_a_grid=numpy.array(SOURCE_PHI_GRID, dtype=numpy.float64), 
    phi_b_grid=numpy.array(SOURCE_PHI_GRID, dtype=numpy.float64)
)

C_DATA_SI = NS.spectra(
    factor=numpy.array(FACTOR_SI, dtype=numpy.float64), 
    amplitude=numpy.array(AMPLITUDE_SI, dtype=numpy.float64), 
    chi_grid=numpy.array(CHI_GRID, dtype=numpy.float64), 
    power_grid=numpy.array(POWER_GRID * ALIGNMENT, dtype=numpy.float64), 
    redshift_grid=numpy.array(Z_GRID, dtype=numpy.float64), 
    phi_a_grid=numpy.array(SOURCE_PHI_GRID, dtype=numpy.float64), 
    phi_b_grid=numpy.array(SOURCE_PHI_GRID, dtype=numpy.float64)
)

C_DATA_IS = NS.spectra(
    factor=numpy.array(FACTOR_IS, dtype=numpy.float64), 
    amplitude=numpy.array(AMPLITUDE_IS, dtype=numpy.float64), 
    chi_grid=numpy.array(CHI_GRID, dtype=numpy.float64), 
    power_grid=numpy.array(POWER_GRID * ALIGNMENT, dtype=numpy.float64), 
    redshift_grid=numpy.array(Z_GRID, dtype=numpy.float64), 
    phi_a_grid=numpy.array(SOURCE_PHI_GRID, dtype=numpy.float64), 
    phi_b_grid=numpy.array(SOURCE_PHI_GRID, dtype=numpy.float64)
)

C_DATA_II = NN.spectra(
    factor=numpy.array(FACTOR_II, dtype=numpy.float64), 
    amplitude=numpy.array(AMPLITUDE_II, dtype=numpy.float64), 
    chi_grid=numpy.array(CHI_GRID, dtype=numpy.float64), 
    power_grid=numpy.array(POWER_GRID * ALIGNMENT ** 2, dtype=numpy.float64), 
    phi_a_grid=numpy.array(SOURCE_PHI_GRID, dtype=numpy.float64), 
    phi_b_grid=numpy.array(SOURCE_PHI_GRID, dtype=numpy.float64)
)

In [ ]:
C_CCL_SS[0, 3, :], C_DATA_SS[0, 3, :], C_DATA1_SS[0, 3, :], C_DATA2_SS[0, 3, :], C_DATA3_SS[0, 3, :]

In [ ]:
C_CCL_SI[0, 3, :], C_DATA_SI[0, 3, :], C_DATA1_SI[0, 3, :], C_DATA2_SI[0, 3, :], C_DATA3_SI[0, 3, :]

In [ ]:
C_CCL_IS[3, 0, :], C_DATA_IS[3, 0, :], C_DATA1_IS[3, 0, :], C_DATA2_IS[3, 0, :], C_DATA3_IS[3, 0, :]

In [ ]:
C_CCL_II[0, 3, :], C_DATA_II[0, 3, :], C_DATA1_II[0, 3, :], C_DATA2_II[0, 3, :], C_DATA3_II[0, 3, :]

In [ ]:
C_CCL_EE = C_CCL_SS + C_CCL_SI + C_CCL_IS + C_CCL_II
C_DATA_EE = C_DATA_SS + C_DATA_SI + C_DATA_IS + C_DATA_II
C_DATA1_EE = C_DATA1_SS + C_DATA1_SI + C_DATA1_IS + C_DATA1_II
C_DATA2_EE = C_DATA2_SS + C_DATA2_SI + C_DATA2_IS + C_DATA2_II
C_DATA3_EE = C_DATA3_SS + C_DATA3_SI + C_DATA3_IS + C_DATA3_II

C_RATIO_EE = numpy.divide(C_DATA_EE, C_CCL_EE, out = numpy.ones(C_CCL_EE.shape), where = C_CCL_EE > 0)
C_RATIO1_EE = numpy.divide(C_DATA1_EE, C_CCL_EE, out = numpy.ones(C_CCL_EE.shape), where = C_CCL_EE > 0)
C_RATIO2_EE = numpy.divide(C_DATA2_EE, C_CCL_EE, out = numpy.ones(C_CCL_EE.shape), where = C_CCL_EE > 0)
C_RATIO3_EE = numpy.divide(C_DATA3_EE, C_CCL_EE, out = numpy.ones(C_CCL_EE.shape), where = C_CCL_EE > 0)

In [ ]:
C_CCL_EE[0, 3, :], C_DATA_EE[0, 3, :], C_DATA1_EE[0, 3, :], C_DATA2_EE[0, 3, :], C_DATA3_EE[0, 3, :]

In [ ]:
# Figure
os.environ['PATH'] = '/pscratch/sd/y/yhzhang/texlive/2025/bin/x86_64-linux:' + os.environ['PATH']
pyplot.rcParams['text.latex.preamble'] = r'\usepackage{amsmath}'
pyplot.rcParams['pgf.texsystem'] = 'pdflatex'
pyplot.rcParams['text.usetex'] = True
pyplot.rcParams['font.size'] = 30

FIGURE = pyplot.figure(figsize = (SOURCE_BIN_SIZE * 6, SOURCE_BIN_SIZE * 6))
GRIDSPEC = GridSpec(6 * SOURCE_BIN_SIZE, 6 * SOURCE_BIN_SIZE, figure=FIGURE, wspace=0.0, hspace=0.0)

for (BIN_INDEX1, BIN_INDEX2) in product(range(SOURCE_BIN_SIZE), range(SOURCE_BIN_SIZE)):
    PLOT = FIGURE.add_subplot(GRIDSPEC[6 * BIN_INDEX1: 6 * BIN_INDEX1 + 3, 6 * BIN_INDEX2: 6 * BIN_INDEX2 + 6])
    
    PLOT.plot(ELL_GRID, C_CCL_EE[BIN_INDEX1, BIN_INDEX2,:], color='black', linestyle='-', linewidth=2.5, label=r'$\mathrm{CCL}$')
    
    PLOT.plot(ELL_GRID, C_DATA1_EE[BIN_INDEX1, BIN_INDEX2,:], color='blue', linestyle='-', linewidth=2.5, label=r'$\mathrm{Numeric \: 1st}$')
    
    PLOT.plot(ELL_GRID, C_DATA2_EE[BIN_INDEX1, BIN_INDEX2,:], color='green', linestyle='-', linewidth=2.5, label=r'$\mathrm{Numeric \: 2nd}$')
    
    PLOT.plot(ELL_GRID, C_DATA3_EE[BIN_INDEX1, BIN_INDEX2,:], color='purple', linestyle='-', linewidth=2.5, label=r'$\mathrm{Numeric \: 3rd}$')
    
    PLOT.plot(ELL_GRID, C_DATA_EE[BIN_INDEX1, BIN_INDEX2,:], color='orange', linestyle='-', linewidth=2.5, label=r'$\mathrm{LimberCloud}$')
    
    PLOT.text(x = 500, y = 10**-9, s = r'$a = {:.0f}$'.format(BIN_INDEX1 + 1), fontsize = 25)
    PLOT.text(x = 500, y = 10**-10, s = r'$b = {:.0f}$'.format(BIN_INDEX2 + 1), fontsize = 25)
    
    PLOT.set_yscale('log')
    PLOT.set_xscale('log')
    PLOT.set_xlim(ELL1, ELL2)
    PLOT.set_ylim(10**-12.5, 10**-6.0)
    
    if BIN_INDEX2 > 0: 
        PLOT.set_yticklabels([])
    PLOT.set_xticklabels([])
    PLOT.get_yticklabels()[0].set_visible([])
    
    PLOT = FIGURE.add_subplot(GRIDSPEC[6 * BIN_INDEX1 + 3: 6 * BIN_INDEX1 + 6, 6 * BIN_INDEX2: 6 * BIN_INDEX2 + 6])
    
    PLOT.plot(ELL_GRID, numpy.ones(ELL_SIZE + 1), color='black', linestyle='-', linewidth=2.5)
    
    PLOT.plot(ELL_GRID, numpy.ones(ELL_SIZE + 1) * (1.00 + 0.01), color = 'grey', linestyle = '--', linewidth = 2.5)
    
    PLOT.plot(ELL_GRID, numpy.ones(ELL_SIZE + 1) * (1.00 - 0.01), color = 'grey', linestyle = '--', linewidth = 2.5)
    
    PLOT.plot(ELL_GRID, C_RATIO1_EE[BIN_INDEX1, BIN_INDEX2,:], color='blue', linestyle='-', linewidth=2.5)
    
    PLOT.plot(ELL_GRID, C_RATIO2_EE[BIN_INDEX1, BIN_INDEX2,:], color='green', linestyle='-', linewidth=2.5)
    
    PLOT.plot(ELL_GRID, C_RATIO3_EE[BIN_INDEX1, BIN_INDEX2,:], color='purple', linestyle='-', linewidth=2.5)
    
    PLOT.plot(ELL_GRID, C_RATIO_EE[BIN_INDEX1, BIN_INDEX2,:], color='orange', linestyle='-', linewidth=2.5)
    
    PLOT.set_xscale('log')
    PLOT.set_xlim(ELL1, ELL2)
    PLOT.set_ylim(0.975, 1.025)
    
    if BIN_INDEX1 < SOURCE_BIN_SIZE - 1: 
        PLOT.set_xticklabels([])
    if BIN_INDEX2 > 0: 
        PLOT.set_yticklabels([])
    PLOT.get_xticklabels()[0].set_visible([])

LEGEND = [
    mlines.Line2D([], [], color='black', linestyle='-', linewidth=2.5, label=r'$\mathrm{CCL}$'),
    mlines.Line2D([], [], color='blue', linestyle='-', linewidth=2.5, label=r'$\mathrm{Numeric \: 1st}$'),
    mlines.Line2D([], [], color='green', linestyle='-', linewidth=2.5, label=r'$\mathrm{Numeric \: 2nd}$'),
    mlines.Line2D([], [], color='purple', linestyle='-', linewidth=2.5, label=r'$\mathrm{Numeric \: 3rd}$'),
    mlines.Line2D([], [], color='orange', linestyle='-', linewidth=2.5, label=r'$\mathrm{LimberCloud}$')
]
FIGURE.legend(handles=LEGEND, loc='upper left', fontsize=25)

FIGURE.supxlabel(r'$\ell$', fontsize=50)
FIGURE.supylabel(r'$C^{a b}_{\epsilon \epsilon} \left(\ell\right)$', fontsize = 50)

FIGURE.subplots_adjust(hspace=0.0, wspace=0.0)
FIGURE.savefig(os.path.join(PLOT_FOLDER, 'CELL_EE.pdf'), bbox_inches='tight')